In [4]:
# from pyspark.sql.functions import * とする場合もありますが、
# Fで関数の名前空間を明示した方がわかりやすくて好きです。
# ただ、FだとPEP8に違反していますが。。。
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, TimestampType, StringType
from pyspark.sql.window import Window
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pandas as pd
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import japanize_matplotlib

# spark initialization
spark = SparkSession.builder.appName("gamedata").getOrCreate()

In [5]:
df = spark.read.parquet("../transform/hard_spark")

In [10]:
df.schema

StructType(List(StructField(begin_date,StringType,true),StructField(end_date,StringType,true),StructField(hw,StringType,true),StructField(units,StringType,true)))

22/05/31 00:00:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 256571 ms exceeds timeout 120000 ms
22/05/31 00:00:13 WARN SparkContext: Killing executors is not supported by current scheduler.


In [20]:
df_hw = df.select("hw").distinct()

In [21]:
for r in df_hw.collect():
    print(r['hw'])

df_hw.count()

PS3
PS
Switch
Vita
DC
PS2
3DS
WiiU
GB
PKS
N64
XBOne
PSP
PS4
XB360
XSX
GBA
WS
SATURN
GC
Wii
PS5
Xbox
DS
NeoGeoP


25

In [28]:
df_ps3 = df.select(
    F.col("begin_date").cast("date"),
    F.col("end_date").cast("date"),
    F.col("units").cast("int").alias("PS3")).where(df.hw == 'PS3').show()

+----------+----------+-----+
|begin_date|  end_date|  PS3|
+----------+----------+-----+
|2006-11-06|2006-11-12|88443|
|2006-11-13|2006-11-19|43378|
|2006-11-20|2006-11-26|37261|
|2006-11-27|2006-12-03|30467|
|2006-12-04|2006-12-10|43597|
|2006-12-11|2006-12-17|70557|
|2006-12-18|2006-12-24|76506|
|2006-12-25|2006-12-31|76506|
|2007-01-01|2007-01-07|67620|
|2007-01-08|2007-01-14|33492|
|2007-01-15|2007-01-21|25379|
|2007-01-22|2007-01-28|21028|
|2007-01-29|2007-02-04|20687|
|2007-02-05|2007-02-11|22950|
|2007-02-12|2007-02-18|20617|
|2007-02-19|2007-02-25|21558|
|2007-02-26|2007-03-04|46924|
|2007-03-05|2007-03-11|26925|
|2007-03-12|2007-03-18|19787|
|2007-03-19|2007-03-25|18391|
+----------+----------+-----+
only showing top 20 rows



In [30]:
df_ps4 = df.select(
    F.col("begin_date").cast("date"),
    F.col("end_date").cast("date"),
    F.col("units").cast("int").alias("PS4")).where(df.hw == 'PS4')

In [31]:
df_ps5 = df.select(
    F.col("begin_date").cast("date"),
    F.col("end_date").cast("date"),
    F.col("units").cast("int").alias("PS5")).where(df.hw == 'PS5')

In [38]:
df.withColumn("units2", F.col('units').cast("int") * 2)

DataFrame[begin_date: string, end_date: string, hw: string, units: string, units2: int]

In [41]:
df_u2 = df.select(
    F.col("begin_date").cast("date"),
    F.col("end_date").cast("date"),
    F.lit("hoge").alias("foo"))

In [42]:
df_u2.show()

+----------+----------+----+
|begin_date|  end_date| foo|
+----------+----------+----+
|1998-10-12|1998-10-18|hoge|
|1998-10-12|1998-10-18|hoge|
|1998-10-12|1998-10-18|hoge|
|1998-10-12|1998-10-18|hoge|
|1998-10-19|1998-10-25|hoge|
|1998-10-19|1998-10-25|hoge|
|1998-10-19|1998-10-25|hoge|
|1998-10-19|1998-10-25|hoge|
|1998-10-26|1998-11-01|hoge|
|1998-10-26|1998-11-01|hoge|
|1998-10-26|1998-11-01|hoge|
|1998-10-26|1998-11-01|hoge|
|1998-10-26|1998-11-01|hoge|
|1998-11-02|1998-11-08|hoge|
|1998-11-02|1998-11-08|hoge|
|1998-11-02|1998-11-08|hoge|
|1998-11-02|1998-11-08|hoge|
|1998-11-02|1998-11-08|hoge|
|1998-11-09|1998-11-15|hoge|
|1998-11-09|1998-11-15|hoge|
+----------+----------+----+
only showing top 20 rows



In [43]:
df_not_switch = df.select('*').where(df.hw != 'Switch')


In [48]:
df_new_ps = df.select('*').where(df.hw.like('PS%'))

In [63]:
df_ps5.select(F.sum(F.col("PS5")).alias("PS5_sum"), F.count("PS5").alias("Weeks"), F.avg("PS5"), F.max("PS5"), F.min("PS5")).show()

+-------+-----+----------+--------+--------+
|PS5_sum|Weeks|  avg(PS5)|max(PS5)|min(PS5)|
+-------+-----+----------+--------+--------+
|1635115|   80|20438.9375|  118085|    1133|
+-------+-----+----------+--------+--------+

